In [1]:
from flask import *
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util


In [2]:
model = "malpytest7"

In [3]:
configs = config_util.get_configs_from_pipeline_file(os.path.join('models', model, 'pipeline.config'))
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join('models', model, 'ckpt-4')).expect_partial()

category_index = label_map_util.create_category_index_from_labelmap(os.path.join('annotations', 'label_map.pbtxt'))

In [4]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [5]:
def rec(IMAGE_PATH):
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.4,
                agnostic_mode=False)

    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.savefig("mygraph.png")
    outcome = detections["detection_classes"][0]+1
    #print(outcome)
    return outcome
    #plt.show()

In [6]:
UPLOAD_FOLDER = "../client/src/uploads"
ALLOWED_EXTENSIONS = {'png', 'jpeg', 'jpg'}
app = Flask(__name__, static_folder='../client/src')
CORS(app)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

In [7]:
def allowed_file(filename):
    return '.' in filename and \
        filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [8]:
@app.route("/")
def home():
    return "hello world"

In [9]:
@app.route("/upload", methods=["POST", "GET"])
def upload():
    if request.method == 'POST':
        if 'file' not in request.files:
            data = {"message": 0, "monkey" : 0}
            return jsonify(data)
        file = request.files['file']
        if file.filename == '':
            data = {"message": 0, "monkey" : 0}
            return jsonify(data)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(path)
            monkey_id = rec(path)
            print(monkey_id)
            data = {"message": 1, "monkey" : int(monkey_id)}
            return jsonify(data)

In [ ]:
if __name__ == "__main__":
    app.run(port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
127.0.0.1 - - [17/Jan/2023 10:53:01] "OPTIONS /upload HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 10:53:11] "POST /upload HTTP/1.1" 200 -


1


127.0.0.1 - - [17/Jan/2023 10:53:26] "OPTIONS /upload HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 10:53:28] "POST /upload HTTP/1.1" 200 -


5


...

5


127.0.0.1 - - [17/Jan/2023 10:55:08] "POST /upload HTTP/1.1" 200 -


5


127.0.0.1 - - [17/Jan/2023 10:55:14] "OPTIONS /upload HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 10:55:15] "POST /upload HTTP/1.1" 200 -


1
